In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ubuntu/mesolitica-tpu.json'

In [2]:
import string

char_vocabs = [''] + list(string.ascii_lowercase + string.digits) + [' ']

In [3]:
sr = 16000
maxlen = 18
maxlen_subwords = 100
minlen_text = 1
global_count = 0

In [4]:
from google.cloud import storage
import numpy as np
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [5]:
import pandas as pd

df = pd.read_csv('train/TRANS.txt', sep = '\t')
files = df.iloc[:,[0,2]].values.tolist()

In [6]:
import tensorflow as tf
import malaya_speech
from tqdm import tqdm

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [7]:
import re

def loop(files):
    import pinyin
    
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            t = pinyin.get(s[1], format="strip", delimiter=" ")
            splitted = s[0].split('_')
            folder = '_'.join(splitted[:-1])
            f = os.path.join('train', folder, s[0])
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(f)
            if (len(y) / sr) > maxlen:
                continue
            
            t = ''.join([c if c in char_vocabs else ' ' for c in t])
            t = re.sub(r'[ ]+', ' ', t).strip()
            
            new_t = [char_vocabs.index(c) for c in t]
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': new_t, 
                                  'targets_length': [len(new_t)],
                                 'lang': [2]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'mandarin/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [8]:
# loop((files[:100], 0))

In [ ]:
import mp

batch_size = 5000
for i in range(0, len(files), batch_size):
    batch = files[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 8, returned = False)
    global_count += 1

 49%|████▉     | 305/625 [00:09<00:08, 36.11it/s]

Empty generated field: ('targets', [])


 87%|████████▋ | 543/625 [00:13<00:01, 45.83it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 46%|████▌     | 288/625 [00:08<00:10, 33.16it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 625/625 [00:16<00:00, 37.70it/s]


 94%|█████████▍| 588/625 [00:19<00:00, 43.74it/s]

Empty generated field: ('targets', [])


...

 30%|██▉       | 186/625 [00:04<00:09, 44.61it/s]

Error opening 'train/5_2595/5_2595_20170701171101.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171115.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171126.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171136.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171146.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171155.wav': System error.


 39%|███▉      | 243/625 [00:04<00:10, 35.92it/s]

Error opening 'train/5_2595/5_2595_20170701171206.wav': System error.
Error opening 'train/5_2595/5_2595_20170701171220.wav': System error.


 80%|████████  | 503/625 [00:15<00:04, 29.96it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

